In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import simulate
import seaborn as sns
import calculate_return as cr
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 1000)
plt.style.use('seaborn')

Fetching Data

In [ ]:
ports = cr.fetch_returns(sim_type = 'garch', random_seeds = range(10000))

Calculating Quantiles

In [ ]:
y = ports[["dual_phase","single_phase","100","9050"]].sort_index()
y = y.loc[(slice(None), '2070-01-01'),:]
quantile=y.quantile([0.1,0.25,0.5,0.75,0.9,0.99],axis=0).round(decimals=1)
quantile['dual_phase']=quantile['dual_phase'].apply('{:,}'.format)
quantile['100']=quantile['100'].apply('{:,}'.format)
quantile['9050']=quantile['9050'].apply('{:,}'.format)
print(quantile)

Plottin Data

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

Density plots

In [ ]:
fige=y.copy()
fige['dual_phase'] = fige['dual_phase'].div(1000000) #Convert into millions
fige['single_phase'] = fige['single_phase'].div(1000000) #Convert into millions
fige['100'] = fige['100'].div(1000000) #Convert into millions
fige['9050'] = fige['9050'].div(1000000) #Convert into millions
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
sns.kdeplot(data=fige['dual_phase'],clip=(-1, 15),label='Dual Phase')
sns.kdeplot(data=fige['single_phase'],clip=(-1, 15),label='Single Phase')
sns.kdeplot(data=fige['100'],clip=(-1, 15),label='All stock')
sns.kdeplot(data=fige['9050'],clip=(-1, 15),label='Life cycle')
ax.set_xlabel('Returns (MM)') 
ax.ticklabel_format(style='plain', axis='x')
ax.set_ylabel('Probability Density',loc='top', rotation = 0, labelpad = -110)
plt.savefig("Empirical/density.png", bbox_inches = 'tight', pad_inches = 0.1)

Boxplots

In [ ]:
x = ports[["dual_phase","single_phase","100","9050"]]
x = x.rename(columns={'dual_phase':'Dual Phase','single_phase':'Single Phase','100': 'All stock', '9050': 'Life cycle'})
x = x.loc[(slice(None), '2070-01-01'),:]
x = pd.melt(x)
x['value'] = x['value'].div(1000000) #Convert into millions
f, ax = plt.subplots(figsize=(10,10))
ax = sns.boxplot(x='variable',y='value',data=x,showfliers = False,linewidth=2.5)
ax.set_ylabel("Terminal Value (MM)", rotation=0, loc='top', labelpad=-120) 
plt.xlabel("Strategy")
plt.ticklabel_format(style='plain', axis='y',useOffset=True, )
plt.savefig("Empirical/boxplot.png", bbox_inches = 'tight', pad_inches = 0.1)

### Examining Default risk

Finding defaulted investors and the timeseries for them

In [ ]:
z = ports[['savings','cash','total_debt','pv_u','tv_u','dual_phase','single_phase','SU_debt','Nordnet_debt','market_returns']]
zz = z.loc[(slice(None), '2070-01-01'),:]
zzz = zz[zz['tv_u'] <= 0].reset_index()#.sample(n=5)
defaulted=zzz['random_state'].tolist()
defaulted_df=z.loc[defaulted,:]
data=defaulted_df[['dual_phase','total_debt','market_returns', 'tv_u']].reset_index().set_index('period')
data=data.groupby(['period']).mean()

Plotting defaulted investors

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(data[['dual_phase','total_debt', 'tv_u']])
ax2.plot(data[['market_returns']],color='b')
ax1.set_xlabel('period')
ax1.set_ylabel('DKK')
ax2.set_ylabel('Monthly returns')
ax2.set_ylim(-0.1,0.1)
plt.show()